## 이 출처 따라서 yolov3 만들어보기
https://wolfy.tistory.com/259

In [4]:
import sys

from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler

from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## utils
import numpy as np
import random, math, time
# 노트북일 때만 사용
from tqdm.notebook import tqdm

## File Loader
import os#, xmltodict
import os.path as pth
from PIL import Image

# Draw Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## Transformer
from random import sample
import albumentations as A
from albumentations.pytorch.transforms import ToTensor

# Seed
random.seed(53)

In [81]:
# configs
path = 'C:/Users/JungHyeona/Documents/deeplearning_seminar/Biometry_20200722'
root_dir = '/train'
annot_f = './{}/VOCdevkit/VOC2007/Annotations'
image_f = path + '/train/train_img/{}/{}'

classes = ['AC', 'FL', 'HC', 'HUM']

num_classes = len(classes)
feature_size = 7
num_bboxes = 2

## Dataframe

In [12]:
import pandas as pd

In [89]:
x_y = pd.read_csv('./train/info/img_info(x,y,w,h).csv')
minmax = pd.read_csv('./train/info/labelCoordinate_minmax.csv')

In [91]:
x_y.head()

,image_name,class_name,X,Y,W,H
0,20160307_E0000776_I0068889.png,AC,376,108,289,274
1,20160307_E0000776_I0068893.png,AC,317,130,296,254
2,20160307_E0000777_I0069006.png,AC,245,62,408,325
3,20160307_E0000777_I0069020.png,AC,397,194,168,165
4,20160307_E0000778_I0069096.png,AC,367,121,311,283


In [93]:
minmax.head()

,image_name,X_min,Y_min,X_max,Y_max
0,20160307_E0000776_I0068889.png,376,108,665,382
1,20160307_E0000776_I0068893.png,317,130,613,384
2,20160307_E0000777_I0069006.png,245,62,653,387
3,20160307_E0000777_I0069020.png,397,194,565,359
4,20160307_E0000778_I0069096.png,367,121,678,404


## utils

In [102]:
def draw_image(x_y, minmax, w=320, h=320, transforms=None):
    im = np.array(Image.open(image_f.format('AC', df['image_name'][0])).resize((w,h)), dtype=np.uint8)

    # Create figure and axes
    fig, ax = plt.subplots(1, figsize=(7,7))

    xmin = minmax['X_min']
    xmax = minmax['X_max']
    ymin = minmax['Y_min']
    ymax = minmax['Y_max']
    la = x_y['class_name']

    if transforms:
        sample = transforms(image=im, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax, category_ids=la)
        im = sample['image'].permute(1,2,0).numpy()
        xmin = sample['X_min']
        xmax = sample['X_max']
        ymin = sample['Y_min']
        ymax = sample['Y_max']
        la = sample['category_ids']

    # Display the image
    ax.imshow(im)


    # Create a Rectangle patch
    #for l in zip(la):
    # top left (x, y) , (w, h)
    rect = patches.Rectangle((x_y['X'], x_y['Y']), x_y['W'], x_y['H'], linewidth=1, edgecolor='r', facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
    props = dict(boxstyle='round', facecolor='red', alpha=0.9)
    plt.text(x_y['X'], x_y['Y'], classes[0], fontsize=10, color='white')
    plt.axis('off')
    plt.show()

In [110]:
draw_image(x_y, minmax)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## image와 box, label정보를 가져와주는 함수

In [ ]:
# dataset # VOC
def get_infos(annot_f=annot_f, mode='train'):
    annot_dir = annot_f.format(mode)
    result = []
    for ano in [pth.join(annot_dir, ano) for ano in os.listdir(annot_dir)]:
        f = open(ano)
        info = xmltodict.parse(f.read())['annotation']
        image_id = info['filename']
        image_size = np.asarray(tuple(map(int, info['size'].values()))[:2], np.int16)
        w, h = image_size
        box_objects = info['object']
        labels = []
        bboxs = []
        for obj in box_objects:
            try:
                labels.append(classes.index(obj['name'].lower()))
                bboxs.append(tuple(map(int, obj['bndbox'].values())))
            except: pass

        # Resizing Box, Change x1 y1 x2 y2
        # albumentations (normalized box)
        bboxs = np.asarray(bboxs, dtype=np.float64)
        try:
            bboxs[:, [0,2]] /= w
            bboxs[:, [1,3]] /= h
        except: pass
        if bboxs.shape[0] or mode=='test':
            result.append({'image_id':image_id, 'image_size':image_size, 'bboxs':bboxs, 'labels':labels})

    return result
    
trval_list = get_infos()
test_list = get_infos(mode='test')

len(trval_list), len(test_list)